[DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020,2024 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Disease Tracking Dashboard

This is a template for your DIY Disease Tracking Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [33]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json

In [34]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [10]:
import json

# Example data
data = [
    {"Date": "2023-01-01", "Confirmed": 1000, "Deaths": 50, "Recovered": 900},
    {"Date": "2023-01-02", "Confirmed": 1050, "Deaths": 52, "Recovered": 920},
    {"Date": "2023-01-03", "Confirmed": 1100, "Deaths": 53, "Recovered": 950},
    {"Date": "2023-01-04", "Confirmed": 1200, "Deaths": 55, "Recovered": 980},
    {"Date": "2023-01-05", "Confirmed": 1250, "Deaths": 56, "Recovered": 1000},
    {"Date": "2023-01-06", "Confirmed": 1300, "Deaths": 58, "Recovered": 1020}
]

# Write the data to a .json file
with open('initial_data.json', 'w') as f:
    json.dump(data, f, indent=4)

print("Data has been saved to initial_data.json")


Data has been saved to initial_data.json


## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [11]:
import pandas as pd

def wrangle_data(raw_data):
    """
    This function takes raw data (as a list of dictionaries) and returns a cleaned DataFrame.
    It will:
        - Convert date strings to datetime objects.
        - Ensure numerical columns are of the correct type.
        - Handle missing values (if any).
    """
    # Convert raw data to DataFrame
    df = pd.DataFrame(raw_data)
    
    # Convert 'Date' to datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Ensure numerical columns are of the correct type (int)
    df['Confirmed'] = pd.to_numeric(df['Confirmed'], errors='coerce').fillna(0).astype(int)
    df['Deaths'] = pd.to_numeric(df['Deaths'], errors='coerce').fillna(0).astype(int)
    df['Recovered'] = pd.to_numeric(df['Recovered'], errors='coerce').fillna(0).astype(int)
    
    # Handle any missing values if required (e.g., filling with zeros or forward fill)
    # Example: Fill missing numerical values with zero (if applicable)
    # df.fillna(0, inplace=True)
    
    return df

# Test the wrangle function with the sample raw data
raw_data = [
    {"Date": "2023-01-01", "Confirmed": 1000, "Deaths": 50, "Recovered": 900},
    {"Date": "2023-01-02", "Confirmed": 1050, "Deaths": 52, "Recovered": 920},
    {"Date": "2023-01-03", "Confirmed": 1100, "Deaths": 53, "Recovered": 950},
    {"Date": "2023-01-04", "Confirmed": 1200, "Deaths": 55, "Recovered": 980},
    {"Date": "2023-01-05", "Confirmed": 1250, "Deaths": 56, "Recovered": 1000},
    {"Date": "2023-01-06", "Confirmed": 1300, "Deaths": 58, "Recovered": 1020}
]

# Wrangle the raw data
df = wrangle_data(raw_data)

# Display the cleaned DataFrame
df.head()


,Date,Confirmed,Deaths,Recovered
0,2023-01-01,1000,50,900
1,2023-01-02,1050,52,920
2,2023-01-03,1100,53,950
3,2023-01-04,1200,55,980
4,2023-01-05,1250,56,1000


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [12]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests  # For making API calls

# Initial "canned" data (in case API call fails)
initial_data = [
    {"Date": "2023-01-01", "Confirmed": 1000, "Deaths": 50, "Recovered": 900},
    {"Date": "2023-01-02", "Confirmed": 1050, "Deaths": 52, "Recovered": 920},
    {"Date": "2023-01-03", "Confirmed": 1100, "Deaths": 53, "Recovered": 950},
    {"Date": "2023-01-04", "Confirmed": 1200, "Deaths": 55, "Recovered": 980},
    {"Date": "2023-01-05", "Confirmed": 1250, "Deaths": 56, "Recovered": 1000},
    {"Date": "2023-01-06", "Confirmed": 1300, "Deaths": 58, "Recovered": 1020}
]

# Global variable to hold the current data (either "canned" or fetched from API)
df = pd.DataFrame(initial_data)  # Start with initial canned data

# Wrangle the data into a DataFrame
def wrangle_data(raw_data):
    """Wrangle raw data into a DataFrame."""
    df = pd.DataFrame(raw_data)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Confirmed'] = pd.to_numeric(df['Confirmed'], errors='coerce').fillna(0).astype(int)
    df['Deaths'] = pd.to_numeric(df['Deaths'], errors='coerce').fillna(0).astype(int)
    df['Recovered'] = pd.to_numeric(df['Recovered'], errors='coerce').fillna(0).astype(int)
    return df

# Function to plot the Confirmed cases
def plot_confirmed_cases(df):
    """Plot confirmed cases over time."""
    plt.figure(figsize=(10, 6))
    plt.plot(df['Date'], df['Confirmed'], label='Confirmed Cases', color='b', marker='o')
    plt.title('Confirmed Cases Over Time')
    plt.xlabel('Date')
    plt.ylabel('Confirmed Cases')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to fetch fresh data from the API
def fetch_data_from_api():
    """Fetch fresh data from the UKHSA API (simulated for now)."""
    try:
        # For demonstration, replace this URL with the actual API endpoint
        url = "https://example.com/api/covid-data"
        
        # Simulating the API response (in a real case, you'd use requests.get(url))
        response = {
            "Date": ["2023-01-07", "2023-01-08", "2023-01-09", "2023-01-10"],
            "Confirmed": [1350, 1400, 1450, 1500],
            "Deaths": [59, 60, 61, 62],
            "Recovered": [1050, 1100, 1150, 1200]
        }
        
        # In a real scenario, you'd parse the actual API response here
        if response:
            return wrangle_data(response)
        else:
            raise ValueError("No data received from the API.")
    except Exception as e:
        print(f"Error fetching data: {e}")
        return wrangle_data(initial_data)  # Use canned data in case of error

# Refresh the data and update the plot
def refresh_data(button):
    """Handle refreshing data when the button is clicked."""
    global df
    df = fetch_data_from_api()  # Fetch and wrangle the fresh data
    plot_confirmed_cases(df)  # Plot the updated data
    print("Data refreshed successfully!")  # Give feedback to the user

# Create the "Fetch Data" button
refresh_button = widgets.Button(description="Fetch Data")

# Bind the button to the refresh_data function
refresh_button.on_click(refresh_data)

# Display the button
refresh_button


Button(description='Fetch Data', style=ButtonStyle())

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [13]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests  # For making API calls

# Initial "canned" data (in case API call fails)
initial_data = [
    {"Date": "2023-01-01", "Confirmed": 1000, "Deaths": 50, "Recovered": 900},
    {"Date": "2023-01-02", "Confirmed": 1050, "Deaths": 52, "Recovered": 920},
    {"Date": "2023-01-03", "Confirmed": 1100, "Deaths": 53, "Recovered": 950},
    {"Date": "2023-01-04", "Confirmed": 1200, "Deaths": 55, "Recovered": 980},
    {"Date": "2023-01-05", "Confirmed": 1250, "Deaths": 56, "Recovered": 1000},
    {"Date": "2023-01-06", "Confirmed": 1300, "Deaths": 58, "Recovered": 1020}
]

# Global variable to hold the current data (either "canned" or fetched from API)
df = pd.DataFrame(initial_data)  # Start with initial canned data

# Wrangle the data into a DataFrame
def wrangle_data(raw_data):
    """Wrangle raw data into a DataFrame."""
    df = pd.DataFrame(raw_data)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Confirmed'] = pd.to_numeric(df['Confirmed'], errors='coerce').fillna(0).astype(int)
    df['Deaths'] = pd.to_numeric(df['Deaths'], errors='coerce').fillna(0).astype(int)
    df['Recovered'] = pd.to_numeric(df['Recovered'], errors='coerce').fillna(0).astype(int)
    return df

# Function to plot the data based on selected metric and date range
def plot_graph(df, metric, date_range):
    """Plot the selected metric (Confirmed, Deaths, Recovered) over the chosen date range."""
    filtered_df = df.iloc[date_range[0]:date_range[1]]  # Filter data based on the selected date range
    
    plt.figure(figsize=(10, 6))
    plt.plot(filtered_df['Date'], filtered_df[metric], label=f'{metric} Cases', marker='o')
    
    plt.title(f'{metric} Cases Over Time')
    plt.xlabel('Date')
    plt.ylabel(f'{metric} Cases')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.show()

# Function to provide instructions for using the graph
def show_instructions():
    """Display instructions to the user on how to interact with the graph."""
    print("### Instructions for Interacting with the Graph:")
    print("1. **Select Metric**: Choose the metric (Confirmed, Deaths, or Recovered) you want to visualize.")
    print("2. **Adjust Date Range**: Use the date range slider to choose which portion of the data you want to display.")
    print("3. **Explore**: After selecting the metric and adjusting the date range, the graph will automatically update.")
    print("Use these tools to analyze the trends in the chosen metric over time.")

# Create widgets for user interaction
dropdown_metric = widgets.Dropdown(
    options=['Confirmed', 'Deaths', 'Recovered'],
    value='Confirmed',
    description='Metric:'
)

slider_range = widgets.IntRangeSlider(
    value=[0, 5],  # Default range
    min=0,
    max=len(df) - 1,
    step=1,
    description='Date Range:'
)

# Display the widgets and instructions
show_instructions()

# Create the interactive plot
interact(plot_graph, df=widgets.fixed(df), metric=dropdown_metric, date_range=slider_range)


### Instructions for Interacting with the Graph:
1. **Select Metric**: Choose the metric (Confirmed, Deaths, or Recovered) you want to visualize.
2. **Adjust Date Range**: Use the date range slider to choose which portion of the data you want to display.
3. **Explore**: After selecting the metric and adjusting the date range, the graph will automatically update.
Use these tools to analyze the trends in the chosen metric over time.


interactive(children=(Dropdown(description='Metric:', options=('Confirmed', 'Deaths', 'Recovered'), value='Con…

<function __main__.plot_graph(df, metric, date_range)>

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and License** Remember that if you deploy your dashboard as a Binder it will be publicly accessible. Change the copyright notice and take credit for your work! Also acknowledge your sources and the conditions of the license by including this notice: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."